# Advanced OOP
Here, I follow the LinkedIn Learning course [Advanced Python: Object-Oriented
Programming](https://www.linkedin.com/learning/advanced-python-object-oriented-programming/advanced-object-oriented-programming-oop?resume=false&u=72605090)
by Miki Tebeka and try the code. Here is the [course
repo](https://github.com/LinkedInLearning/advanced-python-object-oriented-programming-4510177)
(it is very good).

## Part 5: Class Creation Utilities

### Class decorators

In [ ]:
serializers = {}  # media_type -> class


class serializer:
    def __init__(self, media_type):
        self.media_type = media_type # 'application/json' is passed as `media_type`
    
    def __call__(self, cls): # takes decorated class as an argument
        # this is the application of the decorator on the class
        if (other := serializers.get(self.media_type)):
            name = other.__name__
            msg = f'{self.media_type} already registered to {name}'
            raise ValueError(msg)
        
        dump = getattr(cls, 'dump', None)
        if not callable(dump):
            name = cls.__name__
            raise ValueError(f'{name} does not have a "dump" method')
        
        serializers[self.media_type] = cls
        return cls


def serialize(out, media_type, objects):
    cls = serializers.get(media_type)
    if cls is None:
        raise ValueError('unknown media type: {media_type!r}')
    serializer = cls(out)
    for obj in objects:
        serializer.dump(obj)



import json

@serializer('application/json')
class JSONSerializer:
    def __init__(self, out):
        self.out = out

    def dump(self, obj):
        json.dump(obj, self.out)
        self.out.write('\n')


import sys

events = [
    {
        'login': 'elliot',
        'action': 'logout',
    },
    {
        'login': 'elliot',
        'action': 'access',
        'uri': 'file:///etc/passwd',
    },
]

print(serializers)
# {'application/json': <class '__main__.JSONSerializer'>}
# now that class is "registered", as they call it in the course

serialize(sys.stdout, 'application/json', events)

{'application/json': <class '__main__.JSONSerializer'>}
{"login": "elliot", "action": "logout"}
{"login": "elliot", "action": "access", "uri": "file:///etc/passwd"}


- more relevant usecases for me are the
  [`_IgnoreWarning`](https://github.com/scikit-learn/scikit-learn/blob/031d2f83b7c9d1027d1477abb2bf34652621d603/sklearn/utils/_testing.py#L118)
  class in scikit-learn that filters warnings that raise in tests, and the
  [`deprecated`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.deprecated.html)
  class decorator, which is especially insightful because it can decorate classes,
  functions and properties (meaning other decorators) ([link to
  code](https://github.com/scikit-learn/scikit-learn/blob/031d2f83b7c9d1027d1477abb2bf34652621d603/sklearn/utils/deprecation.py#L11))


### `namedtuple()`

- has a different metaclass than `type` (but that is not the main point here)
- `namedtuple()` returns a new class created dynamically!
    - this class is still an instance of `type`, but it is created via a factory function that uses `type()` internally
    - the returned class doesn't have a custom metaclass by default, but the way it is constructed differs from a regular class statement

In [2]:
from collections import namedtuple

Room = namedtuple('Room', 'x y')

r1 = Room(1, 2)
print(r1) # it has a nice __repr__

print('len:', len(r1)) # __len__ implemented
print('x:', r1.x) # does not work via __getattr__, but because it is a namedtuple, which 
# creates immutable attributes, it adds properties to the class for each field
print('[0]:', r1[0]) # __getitem__ implemented


Room(x=1, y=2)
len: 2
x: 1
[0]: 1


In [3]:
Room.__dict__

# we can see `_tuplegetter`, which indicates that namedtuple internally functions with
# properties

mappingproxy({'__doc__': 'Room(x, y)',
              '__slots__': (),
              '_fields': ('x', 'y'),
              '_field_defaults': {},
              '__new__': <staticmethod(<function Room.__new__ at 0x7f7621efe700>)>,
              '_make': <classmethod(<function Room._make at 0x7f7621efec00>)>,
              '_replace': <function collections.Room._replace(self, /, **kwds)>,
              '__repr__': <function collections.Room.__repr__(self)>,
              '_asdict': <function collections.Room._asdict(self)>,
              '__getnewargs__': <function collections.Room.__getnewargs__(self)>,
              '__match_args__': ('x', 'y'),
              'x': _tuplegetter(0, 'Alias for field number 0'),
              'y': _tuplegetter(1, 'Alias for field number 1'),
              '__module__': '__main__'})

Since the attributes of a `namedtupe()` are immutable, we can use them as keys in a
dictionary:

In [5]:
from collections import defaultdict

players = defaultdict(list)
players[r1].append('amy')
print(players)
# defaultdict(<class 'list'>, {Room(x=1, y=2): ['amy']})


r2 = r1._replace(x=3) # create a new instance, because namedtuples are immutable
print(r2)
print(r2._asdict())

# %% compare
Range = namedtuple('Range', 'low high')

rng = Range(1, 2)
print(rng == r1)

defaultdict(<class 'list'>, {Room(x=1, y=2): ['amy']})
Room(x=3, y=2)
{'x': 3, 'y': 2}
True
